# State Pattern

The State pattern allows an object to alter its behavior when its internal state changes. The object will appear to change its class.

## Problem

You need an object to behave differently depending on its internal state, and the number of states and state transitions could be substantial, making conditional statements complex and error-prone.

## Solution

Create state-specific classes for each state and delegate the state-specific behavior to these classes. The context maintains a reference to the current state object and delegates all state-specific behavior to it.

In [20]:
from abc import ABC, abstractmethod


# The State interface declares methods that all Concrete States should implement
class State(ABC):
    @abstractmethod
    def handle(self, context) -> None:
        pass

    def __str__(self) -> str:
        return self.__class__.__name__

In [21]:
# Concrete States implement various behaviors associated with a state
class DraftState(State):
    def handle(self, context) -> None:
        print("Draft State: Document is being drafted.")
        print("Draft State: Moving to moderation.")
        context.transition_to(ModerationState())


class ModerationState(State):
    def handle(self, context) -> None:
        print("Moderation State: Document is being reviewed.")
        print("Moderation State: Moving to published state.")
        context.transition_to(PublishedState())


class PublishedState(State):
    def handle(self, context) -> None:
        print("Published State: Document is live and viewable by the public.")
        print("Published State: Can move back to draft for revisions.")
        context.transition_to(DraftState())

In [22]:
# The Context defines the interface of interest to clients
class DocumentContext:
    _state = None

    def __init__(self) -> None:
        self.transition_to(DraftState())

    def transition_to(self, state: State) -> None:
        print(f"Context: Transitioning to {state}")
        self._state = state
        self._state.context = self

    def draft(self) -> None:
        self._state.handle(self)

    def moderation(self) -> None:
        self._state.handle(self)

    def publish(self) -> None:
        self._state.handle(self)

In [23]:
# Client code
if __name__ == "__main__":
    # Create context with initial state
    context = DocumentContext()

    # Simulate document workflow
    print("\n[Document Creation]")
    context.draft()

    print("\n[Document Review]")
    context.moderation()

    print("\n[Document Publication]")
    context.publish()

    print("\n[Document Revision]")
    context.draft()

Context: Transitioning to DraftState

[Document Creation]
Draft State: Document is being drafted.
Draft State: Moving to moderation.
Context: Transitioning to ModerationState

[Document Review]
Moderation State: Document is being reviewed.
Moderation State: Moving to published state.
Context: Transitioning to PublishedState

[Document Publication]
Published State: Document is live and viewable by the public.
Published State: Can move back to draft for revisions.
Context: Transitioning to DraftState

[Document Revision]
Draft State: Document is being drafted.
Draft State: Moving to moderation.
Context: Transitioning to ModerationState


## Benefits

* **Single Responsibility Principle**: Separates state-specific behavior into different classes
* **Open/Closed Principle**: Introducing new states doesn't affect context or existing states
* **Eliminates Conditional Statements**: Replaces state-dependent if/else or switch statements
* **Improved Organization**: State transitions are explicit and centralized in state classes
* **Thread Safety**: State transitions can be atomic, improving thread safety

## Implementation with State Machine Library

Python offers several state machine libraries that can simplify implementing the State pattern. One popular library is `transitions`, which provides a clean API for defining states, transitions, and callbacks.

In [24]:
# Install the transitions library if not already installed
# Uncomment and run this cell if needed
# !pip install transitions

In [29]:
from state_machine import (
    Event,
    InvalidStateTransition,
    State,
    acts_as_state_machine,
    after,
    before,
)


@acts_as_state_machine
class Process:
    created = State(initial=True)
    waiting = State()
    running = State()
    terminated = State()
    blocked = State()
    swapped_out_waiting = State()
    swapped_out_blocked = State()

    wait = Event(
        from_states=(
            created,
            running,
            blocked,
            swapped_out_waiting,
        ),
        to_state=waiting,
    )
    run = Event(from_states=waiting, to_state=running)
    terminate = Event(from_states=running, to_state=terminated)
    block = Event(
        from_states=(
            running,
            swapped_out_blocked,
        ),
        to_state=blocked,
    )
    swap_wait = Event(
        from_states=waiting,
        to_state=swapped_out_waiting,
    )
    swap_block = Event(
        from_states=blocked,
        to_state=swapped_out_blocked,
    )

    def __init__(self, name):
        self.name = name

    @after("wait")
    def wait_info(self):
        print(f"{self.name} entered waiting mode")

    @after("run")
    def run_info(self):
        print(f"{self.name} is running")

    @before("terminate")
    def terminate_info(self):
        print(f"{self.name} terminated")

    @after("block")
    def block_info(self):
        print(f"{self.name} is blocked")

    @after("swap_wait")
    def swap_wait_info(self):
        print(f"{self.name} is swapped out and waiting")

    @after("swap_block")
    def swap_block_info(self):
        print(f"{self.name} is swapped out and blocked")


def transition(proc, event, event_name):
    try:
        event()
    except InvalidStateTransition:
        msg = f"Transition of {proc.name} from {proc.current_state} to {event_name} failed"
        print(msg)


def state_info(proc):
    print(f"state of {proc.name}: {proc.current_state}")


In [30]:
def main():
    RUNNING = "running"
    WAITING = "waiting"
    BLOCKED = "blocked"
    TERMINATED = "terminated"

    p1, p2 = Process("process1"), Process("process2")
    [state_info(p) for p in (p1, p2)]

    print()
    transition(p1, p1.wait, WAITING)
    transition(p2, p2.terminate, TERMINATED)
    [state_info(p) for p in (p1, p2)]

    print()
    transition(p1, p1.run, RUNNING)
    transition(p2, p2.wait, WAITING)
    [state_info(p) for p in (p1, p2)]

    print()
    transition(p2, p2.run, RUNNING)
    [state_info(p) for p in (p1, p2)]

    print()
    [transition(p, p.block, BLOCKED) for p in (p1, p2)]
    [state_info(p) for p in (p1, p2)]

    print()
    [transition(p, p.terminate, TERMINATED) for p in (p1, p2)]
    [state_info(p) for p in (p1, p2)]


if __name__ == "__main__":
    main()

state of process1: created
state of process2: created

process1 entered waiting mode
Transition of process2 from created to terminated failed
state of process1: waiting
state of process2: created

process1 is running
process2 entered waiting mode
state of process1: running
state of process2: waiting

process2 is running
state of process1: running
state of process2: running

process1 is blocked
process2 is blocked
state of process1: blocked
state of process2: blocked

Transition of process1 from blocked to terminated failed
Transition of process2 from blocked to terminated failed
state of process1: blocked
state of process2: blocked
